In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook
import sys, os
sys.path.insert(0,'/global/homes/d/dgct/metatlas/')

sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )

from metatlas.helpers import dill2plots as dp
from metatlas.helpers import metatlas_get_data_helper_fun as ma_data
from metatlas.helpers import rt_corrector as rt_corrector
from metatlas.helpers import chromatograms_mp_plots as cp
from metatlas.helpers import chromplotplus as cpp
from metatlas.helpers import spectralprocessing as sp
from metatlas.helpers import fastanalysis as fa
import metatlas.metatlas_objects as metob
from metatlas.helpers import mzmine_helpers as mzm

import qgrid

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

import time
import dill
import numpy as np
import multiprocessing as mp
import pandas as pd
import copy

import matplotlib.pyplot as plot

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)

In [ ]:
#default ema atlas: EMA_pos%50447%
atlases = metob.retrieve('Atlas',name='SK_EMA_MP-SolapP_POS_20170914-2108-proton',username='*')
# atlases = metob.retrieve('Atlas',name='SK-ZZ-MAIZE-HILIC-NEG-EMA50447',username='*')
names = []
for i,a in enumerate(atlases):
    print(i,a.name,pd.to_datetime(a.last_modified,unit='s'))#len(a.compound_identifications)

In [ ]:
atlas = atlases[0]
print atlas.name
print atlas.username

In [ ]:
dp = reload(dp)
# group_names_to_remove = pd.read_csv('/global/homes/b/bpb/Downloads/pks_files_to_remove.csv')['group'].tolist()

groups = dp.select_groups_for_analysis(name = '%20160818_SK-MP_SolarP_MeOHex1_QE119_ZICHIL-549760%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['NEG','QC','Bl'])#['QC','Blank'])


In [ ]:
output_dir = '/global/homes/d/dgct/fast_analysis/new_parameters_py/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
reload(fa)
fa.filter_and_dump(atlas, groups, output_dir)